In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [2]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [3]:
run=True
project_name = "09_05_2024 Freq Monitors Analysis Divergence"
sim_name = "PEC Spheres L=18 r=5_3 Tidy Example"

In [4]:
a=5/3
lambdas = a/np.array([0.35,0.5]) 

In [5]:
structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=rf"H:\phd stuff\tidy3d\structures\08_13_2024 Luis H5 18.01a\sample 2"
                                            ,direction="z", lambda_range=lambdas,
                                            box_size=18*a,runtime_ps=1e-12,min_steps_per_lambda=30, permittivity=11,
                                           scaling=1,shuoff_condtion=1e-15, verbose=True, 
                                           monitors=["flux"], freqs=25, 
                                           cut_condition=1.0, source="planewave", absorbers=120, use_permittivity=False,
                                           ref_only=True
                                           )

Configured successfully.


In [6]:
# parameters that are constant for all the simulations
wavelength = 3.54
freq0 = structure_1.freq0
grids_pw = 15
npml = 2 * grids_pw
random_seed = 1

# Define grid spec
dl = wavelength / grids_pw
grid_spec = td.GridSpec.auto(min_steps_per_wvl=grids_pw)
    
# Define PML layers, for this we have no PML in x, y but `npml` cells in z
periodic_bc = td.Boundary(plus=td.Periodic(), minus=td.Periodic())
pml = td.Boundary(plus=td.Absorber(num_layers=npml), minus=td.Absorber(num_layers=npml))
boundary_spec = td.BoundarySpec(x=periodic_bc, y=periodic_bc, z=pml)

In [7]:
@dataclass
class SimulationParameters:
    """Stores parameters for a given simulation."""
    f0a : any            # Array of frequencies to scan via FFT (in units of freq0)
    Lx : float                   # Length of slab in x
    Ly : float                   # Length of slab in y
    Lz : float                   # Length of slab in z
    space : float                # Space between PML and slab
    fwidth : float               # Bandwidth of the excitation pulse in Hz
    offset : float               # Gaussian source offset; the source peak is at time t = offset/fwidth
    run_time : float             # Run time of simulation (sec)
    ff0 : float                  # Nominal volume filling fraction, actual filling fraction is lower due to overlap between spheres
    radius : float               # Radius of spheres (um)
    material : str               # type of material to use for spheres. "dielectric" or "PEC"
    subpixel : td.SubpixelSpec   # subpixel smoothening spec to be used
    sim_mode : str               # Mode of simulation ("transmission" or "beam_spreading")
    task_name : str              # Name of the task in tidy3d
    ref_ind : float = None       # Refractive index of the spheres, needed if material == 'dielectric'    
    Nt: int = 1                  # Number of snapshots in the field time dependence monitor

In [8]:
def get_mediums(sim_params: SimulationParameters) -> Tuple[td.Medium, td.Medium]:
    """Get the mediums corresponding to the spheres and the background, respectively."""

    if sim_params.material == "dielectric":
        if sim_params.ref_ind is None:
            raise ValueError("must specify SimulationParameters.ref_ind")            
        ff_appx = 1 - np.exp(-sim_params.ff0)
        medium_spheres = td.Medium(permittivity=sim_params.ref_ind**2)
        medium_out = td.Medium(permittivity=1 + (sim_params.ref_ind**2 - 1) * ff_appx)
    
    elif sim_params.material == "PEC":
        medium_spheres = td.PEC
        medium_out = td.Medium(permittivity=1)  

    else:
        raise ValueError(f"unrecognized 'material' of {sim_params.material}")

    return medium_spheres, medium_out

In [9]:
sim_params = SimulationParameters(
    Lx = 18*a,
    Ly = 18*a,
    Lz = 18*a,
    space = 2 * wavelength,
    radius = (5/3)/2,
    # ff0 = 0.35,
    ff0 = 0.8,
    fwidth = structure_1.freqw,
    offset = 0,
    run_time = 30e-12,
    f0a =structure_1.monitor_freqs,
    # material = "dielectric",
    material = "PEC",
    ref_ind =None,
    # ref_ind = np.sqrt(13),
    subpixel = td.SubpixelSpec(),
    sim_mode = "transmission",
    task_name = "PEC_transmission",
    # task_name = "dielectric_transmission",
)

In [10]:
# # 2. Transmittance simulation (PEC spheres)
# sim_params = SimulationParameters(
#     Lx = 6 * wavelength,
#     Ly = 6 * wavelength,
#     Lz = 2 * wavelength,
#     space = 2 * wavelength,
#     radius = 0.05,
#     ff0 = 0.80,
#     fwidth = freq0 / 7.0,
#     offset = 10.0,
#     run_time = 10e-12,
#     f0a = np.linspace(0.8, 1.2, 201).tolist(),
#     material = "PEC",
#     ref_ind = None,
#     subpixel = td.SubpixelSpec(),
#     sim_mode = "transmission",
#     task_name = "PEC_transmission",
# )

In [11]:
medium_spheres, medium_out = get_mediums(sim_params)

Lx = sim_params.Lx
Ly = sim_params.Ly
Lz = sim_params.Lz
radius = sim_params.radius
space = sim_params.space
run_time = sim_params.run_time
Lz_tot = 2 * space + Lz
sim_size = [Lx, Ly, Lz_tot]
# number of spheres to place = slab volume * nominal_density
expanded_volume = (Lx + 2 * radius) * (Ly + 2 * radius) * (Lz + 2 * radius)
nominal_density = sim_params.ff0 / (4 * np.pi / 3 * radius**3)
num_spheres = int(expanded_volume * nominal_density)
# Randomly position spheres
np.random.seed(random_seed)
sphere_geometries = []
print(f"inserting {num_spheres:2e} spheres")
for i in range(num_spheres):
    position_x = np.random.uniform(-Lx / 2 - radius, Lx / 2 + radius)
    position_y = np.random.uniform(-Ly / 2 - radius, Ly / 2 + radius)
    position_z = np.random.uniform(-Lz / 2 - radius, Lz / 2 + radius)
    sphere_i = td.Sphere(
        center=[position_x, position_y, position_z],
        radius=radius
    )
    sphere_geometries.append(sphere_i)

spheres = td.Structure(
    geometry=td.GeometryGroup(geometries=sphere_geometries),
    medium=medium_spheres
)
# Define effective medium around the slab
box_in = td.Box(center=[0, 0, -Lz / 2 - space], size=[td.inf, td.inf, 2 * space])
box_out = td.Box(center=[0, 0, Lz / 2 + space], size=[td.inf, td.inf, 2 * space])
struct_in = td.Structure(geometry=box_in, medium=medium_out)
struct_out = td.Structure(geometry=box_out, medium=medium_out)
structures = [spheres, struct_in, struct_out]
# Define incident plane wave
gaussian = td.GaussianPulse(
    freq0=freq0,
    fwidth=sim_params.fwidth,
)

if sim_params.sim_mode == "transmission":
    source_size = (td.inf, td.inf, 0)
elif sim_params.sim_mode == "beam_spreading":
    source_size = (0.25, 0.25, 0.0)
else:
    raise ValueError(f"sim_mode of {sim_params.sim_mode} not recognized.")
# angle of polarization w.r.t. to the x axis (x-polarized)
source = td.PlaneWave(
    size=source_size,
    center=(0, 0, -Lz_tot / 2.0 + 0.1),
    source_time=gaussian,
    direction="+",
    pol_angle=0,
)
freqs_fft = (np.array(sim_params.f0a)).tolist()
# Records CW (via FFT) transmitted flux through the slab
freq_monitorT = td.FluxMonitor(
    center=[0.0, 0.0, Lz / 2.0 + space / 2.0],
    size=[td.inf, td.inf, 0],
    freqs=freqs_fft,
    name="freq_monitorT",
)
# Records time-dependent transmitted flux through the slab
time_monitorT = td.FluxTimeMonitor(
    center=[0.0, 0.0, Lz / 2.0 + space / 2.0],
    size=[td.inf, td.inf, 0],
    name="time_monitorT",
)


freq_monitorField = td.FieldMonitor(
                center = (0,0,0),
                size = (
                    Lx,Ly,Lz
                    ),
                    fields=["Ex","Ey","Ez"],
                    freqs =freqs_fft,
                    name="freq_monitorField",
                    
                )
  

N_run_time = int(sim_params.run_time / (0.99 * wavelength / (grids_pw * td.C_0 * np.sqrt(3))))
# Records E-fields at the output surface at Nt equally spaced times from 0 to run_time
spread_monitor = td.FieldTimeMonitor(
    center=[0.0, 0.0, Lz / 2.0 + 2 * wavelength / grids_pw],
    size=[td.inf, td.inf, 0.0],
    start=0.4 * run_time,
    stop=0.9 * run_time,
    interval=int(N_run_time / sim_params.Nt),
    fields=["Ex", "Ey", "Ez"],
    name="spread_monitor",
)

# Records permittivity throughout simulation volume
eps_monitor = td.PermittivityMonitor(
    center=[0.0, 0.0, 0.0],
    size=[td.inf, td.inf, Lz + wavelength],
    freqs=[freq0],
    name="eps_monitor",
)
monitors = [freq_monitorT, time_monitorT,freq_monitorField]


# Define simulation parameters     
sim = td.Simulation(
    size=sim_size,
    grid_spec=grid_spec,
    structures=structures,
    sources=[source],
    monitors=monitors,
    run_time=run_time,
    boundary_spec=boundary_spec,
    shutoff=1e-15,
    subpixel=sim_params.subpixel,
)

inserting 1.047900e+04 spheres


10:11:48 W. Europe Daylight Time WARNING: The monitor 'interval' field was left 
                                 as its default value, which will set it to 1   
                                 internally. A value of 1 means that the data   
                                 will be sampled at every time step, which may  
                                 potentially produce more data than desired,    
                                 depending on the use case. To reduce data      
                                 storage, one may downsample the data by setting
                                 'interval > 1' or by choosing alternative      
                                 'start' and 'stop' values for the time         
                                 sampling. If you intended to use the highest   
                                 resolution time sampling, you may suppress this
                                 warning by explicitly setting 'interval=1' in  
                                 the monitor.                                   

In [12]:
if run:
        id0 = ""
        if False:
            print("running ref...")
            id0 = web.upload(sim.copy(update={"structures":[],"medium":medium_eff}), folder_name=project_name,task_name=sim_name+'_0', verbose=True)
            web.start(task_id = id0)
            web.monitor(task_id=id0,verbose=True)
            add_ref=False

        id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
        web.start(task_id = id)
        web.monitor(task_id=id,verbose=True)
    
        ids = (id0 if id0 else '') +'\n' + id
        incidence_folder = "z_incidence"
        file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}/{sim_name}.txt"
        # Check if the folder exists
        if not os.path.exists( rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}"):
            os.makedirs(rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}")
            print(f"Folder '{project_name}/{incidence_folder}' created successfully.")

        # Open file in write mode
        with open(file_path, "w") as file:
            # Write the string to the file
            file.write(ids)


10:11:50 W. Europe Daylight Time Created task 'PEC Spheres L=18 r=5_3 Tidy      
                                 Example' with task_id                          
                                 'fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c' and
                                 task_type 'FDTD'.

                                 View task using web UI at                      
                                 ]8;id=820669;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=393125;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\task]8;;\
                                 ]8;id=393125;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\Id]8;;\]8;id=820669;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\=]8;;\]8;id=395044;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\fdve]8;;\]8;id=820669;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\-df21814f-f6f8-44d0-b2b1-2bd380c9c89c']8;;\.

Output()

10:11:54 W. Europe Daylight Time status = queued

                                 To cancel the simulation, use                  
                                 'web.abort(task_id)' or 'web.delete(task_id)'  
                                 or abort/delete the task in the web UI.        
                                 Terminating the Python script will not stop the
                                 job running on the cloud.

...

Output()

10:12:45 W. Europe Daylight Time status = postprocess

10:13:01 W. Europe Daylight Time status = success

10:13:02 W. Europe Daylight Time View simulation result at                      
                                 ]8;id=907828;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=230746;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\task]8;;\
                                 ]8;id=230746;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\Id]8;;\]8;id=907828;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\=]8;;\]8;id=345133;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\fdve]8;;\]8;id=907828;https://tidy3d.simulation.cloud/workbench?taskId=fdve-df21814f-f6f8-44d0-b2b1-2bd380c9c89c\-df21814f-f6f8-44d0-b2b1-2bd380c9c89c']8;;\.

Folder '09_05_2024 Freq Monitors Analysis Divergence/z_incidence' created successfully.
